#### Fargate restart python script including user input or programmatic input

The script is designed to automate the process of restarting a service in Amazon ECS Fargate. Here's a breakdown of the different functions in the script:

    check_prerequisites(): This function checks if awscli and boto3 are installed by attempting to import them. If either module is not installed, it prompts the user to confirm whether they want to install them using pip. If the user confirms, it installs the missing module.

    configure_aws_credentials(): This function prompts the user to select between using a named AWS CLI profile or programmatically configuring their AWS credentials. If the user chooses to use a named profile, it prompts the user to enter the name of their profile. If the user chooses to configure their credentials programmatically, it prompts the user to enter their AWS access key ID, secret access key, and preferred AWS region name. It then creates a boto3.Session object with the selected configuration and returns the session.

    test_aws_connectivity(session): This function takes a boto3.Session object as input and tests the AWS connectivity by making a simple call to the describe_instances() method of the EC2 client. If the connection fails, an error message is printed and the script exits.

    select_service_to_restart(session): This function takes a boto3.Session object as input and prompts the user to enter the name of their ECS cluster. It then retrieves a list of all the services in the selected cluster using the list_services() method of the ECS client. It then prompts the user to select a service to restart by entering the number of the service in the list. It then restarts the selected service by calling the update_service() method of the ECS client with the forceNewDeployment=True parameter.

    test_service_restart(session): This function takes a boto3.Session object as input and prompts the user to enter the name of the ECS cluster and the name of the service to test. It then retrieves a list of all the tasks in the selected service using the list_tasks() method of the ECS client. It waits for the tasks to stop by repeatedly calling the describe_tasks() method until the task count is 0. Finally, it confirms that the service has restarted successfully by calling the list_tasks() method again and checking if the task count is greater than 0.

Here's the main program flow of the script:

    Check if AWS CLI and Boto3 are installed using check_prerequisites().

    Configure AWS credentials using configure_aws_credentials().

    Test AWS connectivity using test_aws_connectivity().

    Select a service to restart using select_service_to_restart().

    Test if service restart is successful using test_service_restart().

### Diagram of solution

In [ ]:
                                +----------------------+
                                |   Prerequisites Check |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |   AWS Configuration   |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |  AWS Connectivity Test|
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |   Cluster Selection   |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |   Service Selection   |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |  Restart Confirmation |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |    Service Restart   |
                                +----------------------+
                                            |
                                            |
                                            v
                                +----------------------+
                                |  Service Restart Test|
                                +----------------------+


### Restart Fargate BY FUNCTION For Notebook use

## here is the check_prerequisites() function:

In [ ]:
import os
import sys
import subprocess
import boto3

# Check if AWS CLI and Boto3 are installed
def check_prerequisites():
    """
    Check if AWS CLI and Boto3 are installed.
    If not, prompt the user to install them using pip.
    """
    try:
        subprocess.check_output(["aws", "--version"])
        subprocess.check_output(["pip", "show", "boto3"])
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("Error: AWS CLI and/or Boto3 is not installed.")
        install_prerequisites = input("Do you want to install AWS CLI and Boto3? (y/n): ")
        if install_prerequisites.lower() == "y":
            try:
                subprocess.check_output(["pip", "install", "awscli", "boto3"])
                print("AWS CLI and Boto3 have been installed successfully.")
            except subprocess.CalledProcessError:
                print("Error: Failed to install AWS CLI and/or Boto3.")
        else:
            sys.exit()

This function checks if the AWS CLI and Boto3 are installed on the user's system by trying to run the aws --version and pip show boto3 commands using the subprocess.check_output() method.

If either command fails with a CalledProcessError or FileNotFoundError, it means that the corresponding software is not installed. The function prompts the user to install them using pip by typing 'y' or 'n'.

If the user chooses to install the missing software, the function runs the pip install awscli boto3 command using the subprocess.check_output() method to install the software. If the installation fails, the function prints an appropriate error message.

The AWS CLI calls used in this function are:

    subprocess.check_output(["aws", "--version"]): This call checks if the AWS CLI is installed by running the aws --version command.
    subprocess.check_output(["pip", "show", "boto3"]): This call checks if Boto3 is installed by running the pip show boto3 command.
    subprocess.check_output(["pip", "install", "awscli", "boto3"]): This call installs AWS CLI and Boto3 using the pip install awscli boto3 command.

## configure_aws_credentials

In [ ]:
# Configure AWS credentials
def configure_aws_credentials():
    """
    Configure AWS credentials either by using a named profile or programmatically.
    """
    config_type = input("Choose your AWS credentials configuration type (named profile/programmatic): ")
    if config_type.lower() == "named profile":
        profile_name = input("Enter the name of your AWS CLI profile: ")
        session = boto3.Session(profile_name=profile_name)
    elif config_type.lower() == "programmatic":
        access_key_id = input("Enter your AWS access key ID: ")
        secret_access_key = input("Enter your AWS secret access key: ")
        region_name = input("Enter your preferred AWS region name: ")
        session = boto3.Session(
            aws_access_key_id=access_key_id,
            aws_secret_access_key=secret_access_key,
            region_name=region_name
        )
    else:
        print("Invalid configuration type.")
        configure_aws_credentials()

    return session


This function prompts the user to choose a type of AWS credentials configuration: named profile or programmatic.

If the user selects "named profile", the function prompts the user to enter the name of their AWS CLI profile using the input() function. It then creates a new Boto3 session with the specified named profile using the boto3.Session(profile_name=profile_name) method.

If the user selects "programmatic", the function prompts the user to enter their AWS access key ID, secret access key, and preferred AWS region name using the input() function. It then creates a new Boto3 session with the specified programmatic configuration using the boto3.Session(aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key, region_name=region_name) method.

If the user enters an invalid configuration type, the function prints an error message and calls itself recursively to prompt the user again.

The AWS CLI calls used in this function are all handled internally by Boto3 and include the following AWS APIs:

    boto3.Session(profile_name=profile_name): This call creates a new Boto3 session with the specified named profile.
    boto3.Session(aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key, region_name=region_name): This call creates a new Boto3 session with the specified programmatic configuration.



## test_aws_connectivity

In [ ]:
# Test AWS connectivity
def test_aws_connectivity(session):
    """
    Test AWS connectivity by trying to retrieve information from EC2.
    """
    try:
        ec2_client = session.client("ec2")
        ec2_client.describe_instances()
        print("AWS connectivity test passed.")
    except:
        print("Error: AWS connectivity test failed.")
        sys.exit()


This function tests the connectivity to the AWS service by creating an EC2 client using the session.client("ec2") method of the Boto3 Session object and trying to retrieve information about EC2 instances using the ec2_client.describe_instances() method. If the call succeeds, it prints a message that the AWS connectivity test has passed. If the call fails, it prints an error message and exits the program using sys.exit().

The AWS CLI calls used in this function are:

    session.client("ec2"): This call creates an EC2 client using the Boto3 Session object.
    ec2_client.describe_instances(): This call tries to retrieve information about EC2 instances.

## select_service_to_restart() function:

In [ ]:
# List Fargate instances and select a service to restart
def select_service_to_restart(session):
    """
    List all available ECS clusters and prompt the user to choose one.
    Then list all services in the selected cluster and prompt the user to choose one to restart.
    """
    fargate_client = session.client("ecs")
    clusters = fargate_client.list_clusters()["clusterArns"]
    if len(clusters) == 0:
        print("Error: No ECS clusters found.")
        sys.exit()
    else:
        print("Available ECS clusters:")
        for i, cluster in enumerate(clusters):
            print(f"{i+1}. {cluster.split('/')[-1]}")
        cluster_number = int(input("Enter the number of the cluster to use: "))
        cluster_arn = clusters[cluster_number - 1]
        cluster_name = cluster_arn.split('/')[-1]
        print(f"Selected ECS cluster: {cluster_name}")
    try:
        services = fargate_client.list_services(cluster=cluster_arn)["serviceArns"]
        if len(services) > 0:
            print("Services in the selected cluster:")
            for i, service in enumerate(services):
                print(f"{i+1}. {service.split('/')[-1]}")
            service_number = int(input("Enter the number of the service to restart: "))
            service_arn = services[service_number - 1]
            print(f"Restarting service {service_arn.split('/')[-1]}...")
            response = fargate_client.update_service(cluster=cluster_arn, service=service_arn, forceNewDeployment=True)
            print("Service restarted successfully.")
        else:
            print("Error: No services found in the selected cluster.")
            sys.exit()
    except:
        print("Error: Failed to list services or restart service.")
        sys.exit()


This function lists all available ECS clusters using the fargate_client.list_clusters() method of the ECS client. It prompts the user to choose a specific cluster by entering the corresponding number.

Then it retrieves all services in the selected cluster using the fargate_client.list_services() method of the ECS client. It prompts the user to choose a specific service to restart by entering the corresponding number.

Finally, it restarts the chosen service using the fargate_client.update_service() method of the ECS client with the forceNewDeployment=True parameter.

The function handles various errors that might occur while listing services or restarting a service, and it prints appropriate error messages.

The AWS CLI calls used in this function are:

    fargate_client.list_clusters(): This call lists all available ECS clusters.
    fargate_client.list_services(cluster=cluster_arn): This call lists all services in a particular cluster.
    fargate_client.update_service(cluster=cluster_arn, service=service_arn, forceNewDeployment=True): This call restarts a particular service in a cluster.

## test_service_restart

In [ ]:
# Test service restart
def test_service_restart(session):
    """
    Test service restart by checking the deployment status of the latest task in the selected service.
    Repeat the check until the deployment is active or a timeout occurs.
    """
    fargate_client = session.client("ecs")
    task_arnc = None
    timeout_seconds = 300
    start_time = time.time()
    while True:
        try:
            task = fargate_client.list_tasks(cluster=cluster_arn, serviceName=service_arn)["taskArns"][0]
            if task != task_arnc:
                task_arnc = task
                task_details = fargate_client.describe_tasks(cluster=cluster_arn, tasks=[task_arnc])["tasks"][0]
                task_status = task_details["lastStatus"]
                if task_status == "RUNNING":
                    print("Service restart test passed.")
                    break
                elif task_status == "STOPPED":
                    reason = task_details["stoppedReason"]
                    print(f"Error: Service restart failed. Task stopped with reason: {reason}")
                    break
            else:
                time.sleep(5)
            if time.time() - start_time > timeout_seconds:
                print("Error: Service restart test timed out.")
                break
        except:
            print("Error: Failed to test service restart.")
            sys.exit()


This function tests the service restart by checking the deployment status of the latest task in the selected service. It uses the fargate_client.list_tasks() method of the ECS client to retrieve the latest task in the selected service, and the fargate_client.describe_tasks() method to retrieve details of the task.

It repeats the check until the deployment is active (i.e., the lastStatus of the task is "RUNNING") or a timeout occurs. The timeout value is set to 300 seconds by default, but it can be changed if needed.

The function handles various errors that might occur while testing the service restart, and it prints appropriate error messages.

The AWS CLI calls used in this function are:

    fargate_client.list_tasks(cluster=cluster_arn, serviceName=service_arn): This call lists the tasks in a specific service in a cluster.
    fargate_client.describe_tasks(cluster=cluster_arn, tasks=[task_arnc]): This call describes a specific task in a cluster.

